In [ ]:
from pinecone import Pinecone
import json
from datetime import datetime, timedelta

keys_path = 'keys/'

with open(keys_path+'api_keys.json') as f:
  api_keys = json.loads(f.read())

# load pinecone credentials
pine_key = api_keys['pine_key']
pine_index = api_keys['pine_index']

# initialize pinecone
pc = Pinecone(pine_key)
pine_index = pc.Index(pine_index)

# load telegram credentials
api_id = api_keys['api_id']
api_hash = api_keys['api_hash']
phone = api_keys['phone']

In [ ]:
import asyncio
from telethon import TelegramClient
from telethon.sessions import StringSession

async def main():
    async with TelegramClient(StringSession(), api_id, api_hash) as client:
        print(client.session.save())

await main()  # Directly await the function

In [7]:
datetime.now() - timedelta(days=1)

datetime.datetime(2024, 5, 24, 16, 50, 13, 632940)

In [ ]:
# remove records from index older than N days
days=40
end_date = (datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d')
end_date = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())
# create dump embedding with all zeros of 1536 dimensions
zero_emb = [0]*1536

filter = {
    "date": { "$lte": end_date}
    }

# loop until all records older than 30 days are deleted
num_deleted = 0
while True:
    # get 1K random ids of records older than 30 days
    res = index.query(zero_emb, top_k=1000, include_metadata=False, filter=filter)
    if res['matches'] == []:
        print(f'No more records older than {days} days')
        print(f'Deleted {num_deleted} records older than {days} days')
        print('!!!! FINISHED !!!!')
        break
    ids = [r.id for r in res['matches']]
    delete_response = index.delete(ids=ids)
    if delete_response == {}: 
        num_deleted += len(ids)
        print(f'Deleted {len(ids)} records older than {days} days')